<a href="https://colab.research.google.com/github/Profitah/onlinebookstore/blob/main/%EC%98%A8%EB%9D%BC%EC%9D%B8%EC%84%9C%EC%A0%90_%ED%82%A4%EC%9B%8C%EB%93%9C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# requests beautifulsoup4 wordcloud matplotlib 와 관련된 오류 발생시 이 코드를 실행해보세요.
!pip install requests beautifulsoup4 wordcloud matplotlib # 데이터를 수집, 분석 하기 위해 필요한 도구들을 가져옵니다.

In [ ]:
# 페이지 목록에서 특정 데이터를 추출한 뒤 어떤 키워드가 몇 번 들어갔는지 세어주는 코드입니다. // 현재 온라인 서점 알라딘의 2024년 8월 에세이 랭킹을 분석하고 있습니다.
# 예시 : 알라딘에서 oo출판사가 낸 책들의 제목을 수집하고 어떤 키워드의 책을 많이 냈는지 알아보기

# 도구 끌어오기
import requests
from bs4 import BeautifulSoup
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import re

# 한글 폰트 설정 ( 없으면 워드클라우드 글씨 깨짐)
!apt-get install -y fonts-nanum
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

# !! 분석하고 싶은 곳, 링크 넣기 !!
base_url = "https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID=55889&Year=2024&Month=8&Week=2&page={page}&cnt=1000&SortOrder=1"

# 책 제목을 어떻게 출력할 것인가
page_number = 1 # 1페이지에 있는 책제목 부터
all_titles = [] # [] 안에 넣겠다

while True:
    url = base_url.format(page=page_number)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 현재 페이지를 시작으로 책 제목 추출
    titles = [element.text for element in soup.find_all('a', class_='bo3')] # 분석하려는 공간의 태그와 클래스 이름 적기
    if titles:
        all_titles.extend(titles)
        print(f"페이지 {page_number}에서 찾은 제목들: {titles}")
    else:
        print(f"페이지 {page_number}: 책 제목을 찾지 못했습니다. 링크 혹은 클래스 이름을 제대로 기재했는지 확인해주세요 ")
        break

    page_number += 1

    # 숫자부분에 마지막 페이지 적기
    if page_number > 20:
        break

# 도서 총 목록 출력
total_books = len(all_titles)
print(f"총 {total_books}개의 도서를 분석했습니다.")

# !! 제외할 단어 목록 (상황에 맞게 수정하세요.)!!
exclude_words = ['위한', '바쁜', '빠른', 'Do', 'it!', '검색', '목차에서', '된다!', '바빠',
                 '세트', '초등학생을', '-', '초등', '::', '+', '1', '&', '', '전2권', '전2', ':']

# 키워드가 몇 번 반복되었는지 계산 및 출력
all_words = []
for title in all_titles:
    words = title.split()
    filtered_words = [word for word in words if '[국내도서]' not in word and word not in exclude_words]
    all_words.extend(filtered_words)

word_counts = Counter(all_words)

print("단어 사용 횟수 (내림차순):")
for word, count in word_counts.most_common(10):  # !! 상위 빈도 n개의 단어를 출력할 것인지 적기!!
    print(f"{word}: {count}회 사용됨")

# 워드 클라우드(사진) 생성
wordcloud = WordCloud(font_path=font_path, width=800, height=400, background_color='white').generate_from_frequencies(word_counts)

# 워드 클라우드(사진) 출력
plt.figure(figsize=(10, 5)) # 사진 배경 크기 (현재 10x5)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off') # 축 표시 on 혹은 off
plt.show()
